In [20]:
import pandas as pd
import numpy as np

In [107]:
candidates = pd.read_csv('../data/candidates1.csv', dtype = {'District Number': 'str', 'Cash Raised': 'int', 'Cash Spent': 'int'}) # district needs to be string in order to maintain leading 0, cash should be int for clarity

In [108]:
candidates.head(1)

,Unnamed: 0,Name,Affiliation,Incumbent,Winner,Vote %,Cash Raised,Cash Spent,State,District Number
0,0,Jerry Carl,R,0,1,64.9,1971321,1859349,AL,01


In [109]:
# rename columns and drop old index
candidates = candidates.drop('Unnamed: 0', axis = 1)
candidates.columns = ['candidate_name', 'party', 'incumbent', 'winner', 'perc_vote', 'money_raised', 'money_spent', 'state_name', 'district']

candidates.head(1)

,candidate_name,party,incumbent,winner,perc_vote,money_raised,money_spent,state_name,district
0,Jerry Carl,R,0,1,64.9,1971321,1859349,AL,01


In [110]:
candidates.info() # confirm all types as desired

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   candidate_name  892 non-null    object 
 1   party           892 non-null    object 
 2   incumbent       892 non-null    int64  
 3   winner          892 non-null    int64  
 4   perc_vote       841 non-null    float64
 5   money_raised    892 non-null    int64  
 6   money_spent     892 non-null    int64  
 7   state_name      892 non-null    object 
 8   district        892 non-null    object 
dtypes: float64(1), int64(4), object(4)
memory usage: 62.8+ KB


In [111]:
# add column to concat state and district (reduces groupby complexity in analysis)
candidates['state_dist'] = candidates['state_name'] + candidates['district']

candidates.head(1)

,candidate_name,party,incumbent,winner,perc_vote,money_raised,money_spent,state_name,district,state_dist
0,Jerry Carl,R,0,1,64.9,1971321,1859349,AL,01,AL01


In [112]:
print(candidates['state_name'].nunique())
candidates['state_name'].unique()

# confirms that 50 states are present in the data, and the 2-letter abbreviations are correct

50


array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI',
       'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI',
       'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC',
       'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT',
       'VT', 'VA', 'WA', 'WV', 'WI', 'WY'], dtype=object)

In [113]:
candidates['state_dist'].nunique()

# correct number of districts, matches the number of seats with voting rights as established by US law

435

In [114]:
candidates['candidate_name'].nunique()

# 2 candidates with the same name (Antonio Delgado, just in case it's an issue which I don't expect to happen)

891

In [115]:
candidates['party'].value_counts()

# confirm party affiliation is present and correct

D    423
R    387
I     46
L     21
3     15
Name: party, dtype: int64

In [116]:
candidates['incumbent'].value_counts()

# all rows have a value, change to boolean - 1 for incumbent, 0 for not

0    506
1    386
Name: incumbent, dtype: int64

In [ ]:
# not needed for Sourish's scraped data

# conditions = [
#     candidates['incumbent'] == 'not incumbent',
#     candidates['incumbent'] == 'Incumbent'
# ]

# outputs = [ 0, 1 ]

# res = np.select(conditions, outputs)

# candidates['incumbent_bool'] = pd.Series(res)

In [117]:
candidates['winner'].value_counts()

# there should be 435 winners, but only 431 are indicated in the data

0    461
1    431
Name: winner, dtype: int64

In [118]:
# check perc_votes, any districts with no vote percentages?

null_votes = candidates.loc[candidates['perc_vote'].isnull()]

# no candidates with votes information in these districts: LA05, NY22
# drop those districts since no vote information available, set other null perc_vote to 0

In [119]:
candidates = candidates.loc[~candidates['state_dist'].isin(['LA05', 'NY22'])]

In [120]:
candidates['perc_vote'].fillna(0, inplace = True)

In [121]:
candidates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 883 entries, 0 to 891
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   candidate_name  883 non-null    object 
 1   party           883 non-null    object 
 2   incumbent       883 non-null    int64  
 3   winner          883 non-null    int64  
 4   perc_vote       883 non-null    float64
 5   money_raised    883 non-null    int64  
 6   money_spent     883 non-null    int64  
 7   state_name      883 non-null    object 
 8   district        883 non-null    object 
 9   state_dist      883 non-null    object 
dtypes: float64(1), int64(4), object(5)
memory usage: 75.9+ KB


In [122]:
candidates['winner'].value_counts()

# still need to determine winners for 2 districts

0    452
1    431
Name: winner, dtype: int64

In [123]:
# not sure this is the best way (almost certainly not), but it will work
# new dataframe with just the top % vote per district (keep only state_dist and perc_vote columns)
# merge with original dataframe on state_dist
# the np.select method should then work since the condition is more straightforward

winners = candidates[candidates.groupby(['state_dist'])['perc_vote'].transform('max') == candidates['perc_vote']].rename({'perc_vote': 'perc_vote_max'}, axis = 1)
winners.shape


(435, 10)

In [124]:
# two districts were apparently absolutely evenly split 50/50 - filter the non-winning 50% candidate out of winners
winners = winners.loc[~winners['candidate_name'].isin(['Christy Smith', 'Rita Hart'])]
winners.shape

(433, 10)

In [125]:
candidates = pd.merge(candidates, winners[['perc_vote_max', 'state_dist']], on = 'state_dist')
candidates.head()

,candidate_name,party,incumbent,winner,perc_vote,money_raised,money_spent,state_name,district,state_dist,perc_vote_max
0,Jerry Carl,R,0,1,64.9,1971321,1859349,AL,01,AL01,64.9
1,James Averhart,D,0,0,35.0,80095,78973,AL,01,AL01,64.9
2,Barry Moore,R,0,1,65.3,650807,669368,AL,02,AL02,65.3
3,Phyllis Harvey-Hall,D,0,0,34.6,56050,55988,AL,02,AL02,65.3
4,Mike D Rogers,R,1,1,67.5,1193111,1218564,AL,03,AL03,67.5


In [131]:
# loop to add winner_bool column
# if district in ['IA02', 'CA25'] then winner_bool = winner
# else if perc_vote == perc_vote_max then winner_bool = 1
# else if perc_vote != perc_vote_max then winner_bool = 0

winner_bool = []

for index, row in candidates.iterrows():
    if row['state_dist'] in ['IA02', 'CA25']:
        winner_bool.append(row['winner'])
    elif row['perc_vote'] == row['perc_vote_max']:
        winner_bool.append(1)
    else:
        winner_bool.append(0)

candidates['winner_bool'] = winner_bool
# I was struggling with the same thing in the scraping, look in that notebook for how to make this work

# url_list = []

# for r in range(len(st_rep)):
#     abbr = st_rep.loc[r]['state_abbr']
#     rep_num = st_rep.loc[r]['seats']

#     for d in range(1, rep_num + 1):
#         url = f"{url_base}{abbr}{str(d).zfill(2)}{url_suffix}"
#         url_list.append(url)

In [132]:
candidates['winner_bool'].value_counts()

0    450
1    433
Name: winner_bool, dtype: int64

In [133]:
candidates_winner = candidates.loc[candidates['winner_bool'] == 1]

In [134]:
candidates_winner['state_dist'].value_counts()

AL01    1
NC04    1
OH01    1
ND01    1
NC13    1
       ..
IL10    1
IL09    1
IL08    1
IL07    1
WY01    1
Name: state_dist, Length: 433, dtype: int64

In [85]:
candidates_winner.loc[candidates_winner['state_dist'] == 'IA02']

,candidate_name,party,incumbent,winner,perc_vote,money_raised,money_spent,state_name,district,state_dist,perc_vote_max,winner_bool
327,Rita Hart,D,0,0,50.0,4232992,4114400,IA,02,IA02,50.0,1
328,Mariannette Miller-Meeks,R,0,1,50.0,2174194,1721926,IA,02,IA02,50.0,1


In [135]:
candidates.head()

,candidate_name,party,incumbent,winner,perc_vote,money_raised,money_spent,state_name,district,state_dist,perc_vote_max,winner_bool
0,Jerry Carl,R,0,1,64.9,1971321,1859349,AL,01,AL01,64.9,1
1,James Averhart,D,0,0,35.0,80095,78973,AL,01,AL01,64.9,0
2,Barry Moore,R,0,1,65.3,650807,669368,AL,02,AL02,65.3,1
3,Phyllis Harvey-Hall,D,0,0,34.6,56050,55988,AL,02,AL02,65.3,0
4,Mike D Rogers,R,1,1,67.5,1193111,1218564,AL,03,AL03,67.5,1


In [136]:
candidates.to_csv('../data/candidates_clean.csv', index = False)